<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span></li><li><span><a href="#Linear-regression-basics" data-toc-modified-id="Linear-regression-basics-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Linear regression basics</a></span><ul class="toc-item"><li><span><a href="#A1.-Linearity" data-toc-modified-id="A1.-Linearity-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>A1. Linearity</a></span></li><li><span><a href="#A2.-Mean-independence" data-toc-modified-id="A2.-Mean-independence-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>A2. Mean independence</a></span></li><li><span><a href="#A3.-Homoskedastic-and-uncorrelated-disturbances" data-toc-modified-id="A3.-Homoskedastic-and-uncorrelated-disturbances-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>A3. Homoskedastic and uncorrelated disturbances</a></span></li><li><span><a href="#A4.-Full-rank" data-toc-modified-id="A4.-Full-rank-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>A4. Full rank</a></span></li></ul></li><li><span><a href="#Regression-fitting:-ordinary-least-squares-(OLS)" data-toc-modified-id="Regression-fitting:-ordinary-least-squares-(OLS)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Regression fitting: ordinary least squares (OLS)</a></span><ul class="toc-item"><li><span><a href="#Example-1:-estimating-a-stock's-alpha-and-beta-using-the-market-model" data-toc-modified-id="Example-1:-estimating-a-stock's-alpha-and-beta-using-the-market-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Example 1: estimating a stock's alpha and beta using the market model</a></span></li><li><span><a href="#Example-2:-estimating-a-stock's-alpha-and-beta(s)-using-the-Fama-French-three-factor-model" data-toc-modified-id="Example-2:-estimating-a-stock's-alpha-and-beta(s)-using-the-Fama-French-three-factor-model-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Example 2: estimating a stock's alpha and beta(s) using the Fama-French three-factor model</a></span></li></ul></li><li><span><a href="#Interpreting-regression-results" data-toc-modified-id="Interpreting-regression-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Interpreting regression results</a></span><ul class="toc-item"><li><span><a href="#Coefficients" data-toc-modified-id="Coefficients-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Coefficients</a></span></li><li><span><a href="#Statistical-significance" data-toc-modified-id="Statistical-significance-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Statistical significance</a></span></li><li><span><a href="#The-R-squared-coefficient" data-toc-modified-id="The-R-squared-coefficient-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>The R-squared coefficient</a></span></li><li><span><a href="#Diagnostics-(bottom-of-the-table):" data-toc-modified-id="Diagnostics-(bottom-of-the-table):-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Diagnostics (bottom of the table):</a></span></li></ul></li></ul></div>

# Preliminaries


In [1]:
# Import packages
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr

Load Fama-French factor data:

In [2]:
ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '2012-01-01')[0]/100
ff3f.head(2)

/tmp/ipykernel_10465/2771932554.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '2012-01-01')[0]/100
/tmp/ipykernel_10465/2771932554.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '2012-01-01')[0]/100


,Mkt-RF,SMB,HML,RF
Date,,,,
2012-01,0.0505,0.0204,-0.0097,0.0
2012-02,0.0442,-0.0185,0.0043,0.0


Download monthly prices (keep only Adjusted Close prices):

In [3]:
firm_prices = yf.download('TSLA', '2012-12-01', '2020-12-31', interval = '1mo')['Adj Close'].dropna().to_frame()
firm_prices.head(2)

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
2012-12-01,2.258000
2013-01-01,2.500667


Calculate monthly returns, drop missing and rename "Adj Close" to "TSLA":

In [4]:
firm_ret = firm_prices.pct_change().dropna().rename(columns = {'Adj Close': 'TSLA'})
firm_ret.head(2)

,TSLA
Date,
2013-01-01,0.107470
2013-02-01,-0.071448


We need to merge ``firm_ret`` with ``ff3f`` but note that their dates look different. Check their format first:

In [5]:
firm_ret.index.dtype

dtype('<M8[ns]')

In [6]:
ff3f.index.dtype

period[M]

Convert index of ``firm_ret`` to monthly period, to match the date format un ``ff3f``:

In [7]:
firm_ret.index = firm_ret.index.to_period('M')
firm_ret.head(2)

,TSLA
Date,
2013-01,0.107470
2013-02,-0.071448


Merge the two datasets:

In [8]:
data = firm_ret.join(ff3f)
data.head(2)

,TSLA,Mkt-RF,SMB,HML,RF
Date,,,,,
2013-01,0.107470,0.0557,0.0033,0.0096,0.0
2013-02,-0.071448,0.0129,-0.0027,0.0011,0.0


In [9]:
data['const'] = 1
data

,TSLA,Mkt-RF,SMB,HML,RF,const
Date,,,,,,
2013-01,0.107470,0.0557,0.0033,0.0096,0.0000,1
2013-02,-0.071448,0.0129,-0.0027,0.0011,0.0000,1
2013-03,0.087855,0.0403,0.0081,-0.0019,0.0000,1
2013-04,0.424914,0.0155,-0.0237,0.0045,0.0000,1
2013-05,0.810706,0.0280,0.0170,0.0263,0.0000,1
...,...,...,...,...,...,...
2020-08,0.741452,0.0763,-0.0022,-0.0297,0.0001,1
2020-09,-0.139087,-0.0363,0.0002,-0.0271,0.0001,1
2020-10,-0.095499,-0.0210,0.0438,0.0425,0.0001,1


# Linear regression basics

A linear regression is a statistical model, which means it is a set of assumptions about the relation between two or more variables. In particular, the standard linear regression assumptions are (we restrict ourselves to two variables X and Y for now):

## A1. Linearity 

The relation between the variables is assumed to be linear in parameters:

$$Y_t = \alpha + \beta \cdot X_t + \epsilon_t $$

Note that "linear in parameters" means the function that describes the relation between X and Y (the equation above) is linear with respect to $\alpha$ and $\beta$ (e.g. $Y = \alpha \cdot X^{\beta} + \epsilon$ is not linear in parameters). It does not mean that the relation needs to be linear with respect to X (e.g. $Y = \alpha + \beta \cdot X^2 + \epsilon$ is still linear in parameters).

Before we cover the remaining assumptions, a bit of terminology:


- Y is commonly referred to as the "dependent", or "explained" or "endogenous" variable


- X is commonly referred to as the "independent", or "explanatory", or "exogenous" variable (though, remember, that X can stand for more than one variables)


- $\epsilon$ is commonly referred to as the "residual" of the regression, or the "error" term, or the "disturbance" term


- $\alpha$ (alpha) and $\beta$ (beta) are the "coefficients" or "parameters" of the regression. **The outcome of "running a regression" is to calculate estimates for these alpha and beta coefficients.** 


- the t subscript is meant to represent the fact that we observe multiple realizations of the X and Y variables, and the linear relation is assumed to hold for each set of realizations (different t's can represent different points in time, or different firms, different countries, etc). Going forward in this section, we will assume that t stands for time, to make the interpretation clearer.

## A2. Mean independence

This assumption states that the independent variable(s) X convey no information about the disturbance terms ($\epsilon$'s). Technically, we write this assumption as:

$$E[\epsilon_t | X] = 0$$

This is also called the "strict exogeneity" assumption. When this condition is not satisfied, we say that our regression model has an **endogeneity problem**.


## A3. Homoskedastic and uncorrelated disturbances

This assumption states that all disturbance terms have the same variance (i.e. they are "homoskedastic"): 

$$ Var[\epsilon_t | X] = \sigma^2                 \text{  , for all observations $t$} $$

When this assumption is not satisfied, we say we have a **heteroskedasticity problem**.

The standard linear regression model also assumes that any two disturbance terms are uncorrelated with each other:

$$ Cov [\epsilon_{t_1}, \epsilon_{t_2}] = 0 \text{ , for all $t_1 \neq t_2$}$$ 



## A4. Full rank

This assumption states that there are no exact linear relationships between the explanatory variables X (when there are two or more such variables). When this assumption is not satisfied, we say we have a **multicollinearity problem**. 

In the next few lectures, we will cover strategies that we can use when some of the above assumptions are not satisfied.

# Regression fitting: ordinary least squares (OLS) 

By far the most common method for estimating linear regression coefficients is by minimizing the sum of the squares of the error terms (hence "least squares"). 

The package we will use for linear regression fitting is called "**statsmodels**". Install this package by typing the following in a terminal (or Anaconda Prompt):

**pip install statsmodels**

In fact, for the most part, we will only use the "api" subpackage of "statsmodels" as below. Here is the official documentation for the package if you want to learn more about it:

https://www.statsmodels.org/stable/index.html

In [11]:
import statsmodels.api as sm

As mentioned above, we will estimate our regression coefficients using OLS (ordinary least squares). This can be done with the ``OLS`` function of ``statsmodels``:

Syntax:
```python
class statsmodels.regression.linear_model.OLS(endog, exog=None, missing='none', hasconst=None, **kwargs)
```

When we use this function, we can replace ``statsmodels.regression.lineal_model`` with ``sm`` (as imported above).
The ``endog`` parameter is where we specify where the data for our dependent variable is, and the ``exog`` parameter is where we specify where our independent variables are. We usually set ``missing=True`` to tell Python that we want to get rid of all the rows in our regression data that have any missing values.

## Example 1: estimating a stock's alpha and beta using the market model

The market model (aka the "single-factor model" or the "single-index model") is a linear regression model that relates the excess return on a stock to the excess returns on the market portfolio: 

$$R_{i,t} - R_{f,t} = \alpha_i + \beta_i (R_{m,t} - R_{f,t}) + \epsilon_{i,t}$$

where:
- $R_{i,t}$ is the return of firm $i$ at time $t$
- $R_{m,t}$ is the return of the market at time $t$ (we generally use the S&P500 index as the market portfolio)
- $R_{f,t}$ is the risk-free rate at time $t$ (most commonly the yield on the 1-month Tbill)

Below, we estimate this model for TSLA, using the data we gathered at the top of these lecture notes:

To "run" (i.e. "fit" or "estimate") the regression, we use the ``.fit()`` function which can be applied after the ``sm.OLS()`` function. We store results in "res":

In [12]:
res = sm.OLS(endog = data['TSLA']-data['RF'],
             exog= data[['const', 'Mkt-RF']],
             missing='drop'
             ).fit()
res

The above shows that ``res`` is a "RegressionResultsWrapper". We have not seen this kind of an object before. Check all the attributes of the results (``res``) object:

In [13]:
print(dir(res))

['HC0_se', 'HC1_se', 'HC2_se', 'HC3_se', '_HCCM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abat_diagonal', '_cache', '_data_attr', '_data_in_cache', '_get_robustcov_results', '_get_wald_nonlinear', '_is_nested', '_transform_predict_exog', '_use_t', '_wexog_singular_values', 'aic', 'bic', 'bse', 'centered_tss', 'compare_f_test', 'compare_lm_test', 'compare_lr_test', 'condition_number', 'conf_int', 'conf_int_el', 'cov_HC0', 'cov_HC1', 'cov_HC2', 'cov_HC3', 'cov_kwds', 'cov_params', 'cov_type', 'df_model', 'df_resid', 'eigenvals', 'el_test', 'ess', 'f_pvalue', 'f_test', 'fittedvalues', 'fvalue', 'get_influence', 'get_prediction', 'get_robustcov_results', 'info_criteria', 'initialize', 'k_c

Particularly important results are stored in ``summary()``, ``params``, ``pvalues``, ``tvalues``, ``rsquared``. We'll cover all of these below. As the name suggests, the ``summary()`` attribute contains a summary of the regression results:

In [15]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     17.26
Date:                Wed, 20 Mar 2024   Prob (F-statistic):           7.18e-05
Time:                        09:55:41   Log-Likelihood:                 30.144
No. Observations:                  96   AIC:                            -56.29
Df Residuals:                      94   BIC:                            -51.16
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0416      0.019      2.186      0.0

## Example 2: estimating a stock's alpha and beta(s) using the Fama-French three-factor model

The Fama-French three factor model is a linear regression model that relates the excess return on a stock to the excess returns on the market portfolio and the returns on the SMB (small minus big) and HML (high minus low book-to-market) factors: 

$$R_{i,t} - R_{f,t} = \alpha_i + \beta_{i,m} (R_{m,t} - R_{f,t}) + \beta_{i,smb}  R_{smb} + \beta_{i,hml} R_{hml} + \epsilon_{i,t}$$

**Challenge**:

Estimate this regression for TSLA using the data we gathered at the top of these lecture notes.

In [16]:
# Run the regression and print the results
res3 = sm.OLS(endog = data['TSLA']-data['RF'],
             exog= data[['const', 'Mkt-RF', 'SMB', 'HML']],
             missing='drop'
             ).fit()

print(res3.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     6.045
Date:                Wed, 20 Mar 2024   Prob (F-statistic):           0.000837
Time:                        10:06:32   Log-Likelihood:                 30.689
No. Observations:                  96   AIC:                            -53.38
Df Residuals:                      92   BIC:                            -43.12
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0370      0.020      1.867      0.0

# Interpreting regression results

## Coefficients

The "const" row contains information about the firm's $\alpha$ and the "Mkt-RF" row contains information about the firm's $\beta$. The $\alpha$ and $\beta$ coefficient estimates themselves are in the "coef" column ($\alpha = 0.0416$, and $\beta = 1.83$ in the single-factor model).

The "const" coefficient tells us what we should expect the return on TSLA to be on a day with no systematic shocks (i.e. a day with market return of 0).

The 'Mkt-RF' coefficient tells us how we should expect the return on TSLA to react to a given shock to the market portfolio (e.g. the 1.83 coefficient tells us that, on average, if the market goes up by 1%, TSLA goes up by 1.83%, and when the market goes down by 1%, TSLA goes down by 1.83%)

The results object (``res``) stores the regression coefficients in its ``params`` attribute:

In [17]:
res.params

const     0.041640
Mkt-RF    1.829764
dtype: float64

Note that ``res.params`` is a Pandas Series, so we can access its individual elements using the index labels:

In [19]:
print("Alpha = " , res.params['const'])
print("Beta = " , res.params['Mkt-RF'])

Alpha =  0.041640223340697884
Beta =  1.8297640959417043


**Challenge**: 

Print out (separately) the alpha and each of the betas from the three-factor model

In [21]:
print("Alpha = " , res3.params['const'])
print("Market Beta = " , res3.params['Mkt-RF'])
print("SMB Beta = " , res3.params['SMB'])
print("HML Beta = " , res3.params['HML'])

Alpha =  0.03698299962428881
Market Beta =  1.8843393672489246
SMB Beta =  0.1804910656407513
HML Beta =  -0.6808407269943435


## Statistical significance

**The p-values** are in the "P > |t|" column. P-values lower than 0.05 allow us to conclude that the corresponding coefficient is statistically different from 0 at the 95% confidence level (i.e. reject the null hypothesis that the coefficient is 0). At the 99% confidence level, we would need the p-value to be smaller than 1% (1 minus the confidence level) to reject the null hypothesis that alpha = 0.

**The t-statistics** for the two coefficients are in the "t" column. Loosely speaking a t-statistic that larger than 2 or smaller than -2 allows us to conclude that the corresponding coefficient is statistically different from 0 at the 95% confidence level (i.e. reject the null hypothesis that the coefficient is 0). In terms of statistical significance, the t-statistic does not provide any new information over the p-value.

The last two columns give us the 95% confidence interval for each coefficient.

For the market model, TSLA's alpha has a p-value of 0.031 so we can conclude that it's alpha is statistically significantly different from 0 at the 95% confidence level (but not at the 99% confidence level).

The fact that the alpha is positive and statistically different from 0 (at 95% level) means that, based on the single-factor model, TSLA seems to be **undervalued**. A negative alpha would mean the stock in **overvalued**.

If we can not reject the null hypothesis that alpha is 0, the conclusion is NOT that alpha = 0 and therefore the stock is correctly valued (since we can never "accept" a null hypothesis, we can only fail to reject). The conclusion is that we do not have enough evidence to claim that the stock is either undervalued or overvalued (which is not the same thing as saying that we have enough evidence to claim that the stock is correctly valued).

The results object (``res``) stores the regression p-values in its ``pvalues`` attribute:

In [22]:
res.pvalues

const     0.031321
Mkt-RF    0.000072
dtype: float64

The p-values can be accessed individually:

In [24]:
res.pvalues['const']
res.pvalues['Mkt-RF']

7.180072935034791e-05

T-statistics are stored in the ``tvalues`` attribute:

In [25]:
res.tvalues

const     2.185707
Mkt-RF    4.154461
dtype: float64

T-statistics of individual coefficients:

In [26]:
res.tvalues['const']

2.185707179744499

**Challenge**: 

Is TSLA mispriced (undervalued OR overvalued) at the 5% significance level with respect to the Fama-French 3-factor model?

In [27]:
res3.pvalues['const'] < 0.05

False

**Challenge**: 

Does TSLA have a significant exposure (at 5% level) to either of the 3 factors in the Fama-French model?

In [31]:
res3.pvalues['HML'] < 0.05

False

## The R-squared coefficient 

The R-squared coefficient (top-right of the table, also referred to as the "coefficient of determination") estimates the percentage of the total variance in the dependent variable (Y) that can be explained by the variance in the explanatory variable(s) (X). 

In the context of our market-model example, the R-squared tells us the percentage of the firm's total variance that is systematic in nature (i.e. non-diversifiable). The percentage of total variance that is idiosyncratic (diversifiable) equals 1 minus the R-squared.

The R-squared is stored in the ``rsquared`` attribute of the regression results object:

In [32]:
res.rsquared

0.15512868994096118

Using the market model, our estimates of the percentage or total TSLA variance that is systematic vs idiosyncratic are:

In [34]:
print("Percent of total TSLA variance that is systematic: ", res.rsquared)
print("Percent of total TSLA variance that is idiosyncratic: ", 1 - res.rsquared)

Percent of total TSLA variance that is systematic:  0.15512868994096118
Percent of total TSLA variance that is idiosyncratic:  0.8448713100590388


**Challenge**: 

What percentage of TSLA total variance can be diversified away under the Fama-French 3-factor model?

In [35]:
print("Percent of total TSLA variance that is systematic: ", res3.rsquared)
print("Percent of total TSLA variance that is idiosyncratic: ", 1 - res3.rsquared)

Percent of total TSLA variance that is systematic:  0.1646710904791645
Percent of total TSLA variance that is idiosyncratic:  0.8353289095208355


## Diagnostics (bottom of the table):

The regression table reports (at the bottom) a few statistics that help us understand if some of the assumption of the linear regression model are not satisfied.

- Durbin-Watson: tests for residual autocorrelation. Takes values in [0,4]. Below 2 means positive autocorr. Above 2 means negative aoutocorr. A value of 2 is perfect (means no autocorrelation).


- Omnibus: tests normality of residuals. Prob(Omnibus) close to 0 means reject normality


- JB: another normality test (null is skew=0, kurt=3). Prob(JB) close to 0 means rejection of normality


- Cond. No: tests for multicollinearity. Over 100 is worrisome, but we still have to look at correlations between variables to determine if any of them need to be dropped.